In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

모델 구조

Meta Data 생성  
불균형 데이터 조정  
결측치 처리  
Feature Engineering  
범주형 변수 인코딩  
Mean Encoding  
One-hot Encoding  
PolynomialFeatures  
Feature Selection  
VarianceThreshold  
SelectFromModel  
Feature Scaling  
Modeling  

<대회 설명>  

Porto Seguro는 브라질의 자동차 보험 회사.    
보험 청구할 확률이 높은 차주한테 보험료를 높게 청구하고, 아닌 차주는 낮게 보험료를 청구     

<대회 목적>

어떤 차주가 내년에 보험 청구 할 확률 예측

<데이터 특징>

테스트 데이터 > 훈련 데이터
결측치의 값이 -1로 주어짐
실제 기업 데이터이기 때문에 feature를 비식별화 해놨음 (그렇기 때문에 난이도가 높은 쪽에 속함),  
지니 예상으로는 사고 횟수, 보험같이든 가족 수 등등이 아닐까?         
target = 0 : 보험 청구 no / target = 1 : 보험 청구 yes  
<데이터 평가>

predict_proba라는 함수를 사용해서 값을 예측 (확률 값)  

대회의 평가지표는 특이하게 Normalized Gini Coefficient를 사용함  
지니 계수 : 경제적 불평등을 계수화할 때 주로 사용하는 지표
Normalized Gini Coefficient를 사용하는 이유:  
Imbalanced Class를 평가를 위한 임계값을 어떻게 정하느냐에 따라 예측값이 바뀜  
= ROC 커브로 확인 (면적으로 스코어를 매김) >> 비슷한 목적으로 Gini Coefficient가 쓰임  
= gini = 2 * AUC - 1

0~0.5의 값을 가짐
0.5에 가까울수록 좋은 분석임.



In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 100)

In [16]:
train = pd.read_csv('../input/porto-seguros-safe-driver-prediction-train-data/train.csv')
test = pd.read_csv('../input/porto-seguros-safe-driver-prediction-test-data/test.csv')
# sub = pd.read_csv("/kaggle/input/bike-sharing-demand/sampleSubmission.csv")                   


In [17]:
train.head()

In [18]:
train.tail()

In [20]:
train.shape

In [21]:
train.drop_duplicates()
train.shape

No duplicate rows, so that's fine.

In [22]:
test.shape

In [23]:
test.drop_duplicates()
test.shape

In [25]:
train.info()

# Meta Data
role: input, ID, target  
level: nominal, interval, ordinal, binary  
keep: True or False  
dtype: int, float, str  

In [31]:
data = []
for f in train.columns:
    if f =='target':
        role = 'target'
    elif f == 'id':
        role = 'id'
    else:
        role = 'input'
   
    # if there is the bin string in the f or 'target' column, level is binary    
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    # if trian's columns' dtypes are float, level is float.  
    elif train[f].dtype == float:
        level = 'float'
    elif train[f].dtype == int:
        level = 'ordinal'
        
   # Initialize keep to True for all variables except for id
    keep = True
    if f == 'id':
        keep = False
    

    dtype = train[f].dtype
    
    # DataFrame으로 만들기 위해 리스트에 append하기 전에 딕셔너리 타입으로 만들어주었음
    f_dict={
        'varname':f,
        'roel':role,
        'level':level,
        'keep':keep,
        'dtype': dtype
    }
    
    data.append(f_dict)
    
# 변수의 이름을 인덱스로 하는 데이터프레임을 만들어줌    
meta = pd.DataFrame(data, columns = ['varname', 'role', 'level', ' keep', ' dtype'])
meta.set_index('varname', inplace = True)

In [34]:
data = []
for f in train.columns:
    # Defining the role
    if f == 'target':
        role = 'target'
    elif f == 'id':
        role = 'id'
    else:
        role = 'input'
         
    # Defining the level
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    elif train[f].dtype == float:
        level = 'interval'
    elif train[f].dtype == int:
        level = 'ordinal'
        
    # Initialize keep to True for all variables except for id
    keep = True
    if f == 'id':
        keep = False
    
    # trian set의 컬럼별 원래 dtype을 적어라. 
    dtype = train[f].dtype
    
    # Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname': f,   # f에 들어가 있는 모든 train의 colums들 
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace=True)

In [35]:
meta

Example to extract all nominal variables that are not dropped

In [36]:
meta[(meta.level == 'nominal') & (meta.keep)].index

In [37]:
# Q size 함수를 해주는 이유?
pd.DataFrame({'count' : meta.groupby(['role', 'level'])['role'].size()}).reset_index()

위에서 만들어준 정리를 통해 유형별로 데이터를 어떻게 다룰 것인가를 쉽게 선택할 수 있음
ex. 연속형 변수의 경우 통계적 방법, 범주형 변수의 경우 시각화를 통한 탐색 등

In [38]:
# Q .index를 해주는 이유?
Interval = meta[(meta["level"] == "interval") & (meta["keep"])].index

In [40]:
# describe를 통해 interval 변수들의 통계량을 확인 
train[Interval].describe()